In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [5]:
#import nltk
#nltk.download('stopwords')
#we can also print stopwords without using this.Downloaded means accessible even without internet that's it.

In [6]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
#Data Pre-Processing
#Loading dataset to a pandas dataframe
x=pd.read_csv("C:\\Users\\91824\\Downloads\\train.csv\\train.csv")

In [8]:
x.shape

(20800, 5)

In [9]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [10]:
#print the first 5 rows of the dataframe
x.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
#gives the count of number of missing values in the dataset
#x.isnull()-this gives null or not for every tuple-gives true or false
#it is ok if few values are missing we can drop or replace with null string, but if many values are missing then we will use some imputation methods.
#In this data set we no need to change values as we have enough values.
x.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
#replacing null values with empty string.
x=x.fillna('')

In [13]:
#in this project we are using title and author for accuracy not text as the text is so long

In [14]:
#merging the author name and title and writing in new column
x['content']=x['author']+' '+x['title']

In [15]:
print(x['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [16]:
#seperating data and label
a=x.drop(columns='label',axis=1)#here axis=1 means dropping column if 0 drops row
b=x['label']
print(a)
print(b)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [17]:
#Stemming after this vectorization(converting to numericals) is done then feed into ML model.
port_stem=PorterStemmer() #loaded into variable port_stem

In [18]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)#re-useful for searching paragraph for text,sub-substitutes certain values,removes all except alphabets in content
    stemmed_content=stemmed_content.lower()#converts to lowercase
    stemmed_content=stemmed_content.split()#splits to respective list
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]#removes stopwords
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [19]:
x['content']=x['content'].apply(stemming)

In [20]:
print(x['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [21]:
#seperating the data and label
a=x['content'].values
b=x['label'].values

In [22]:
print(a)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [23]:
print(b)#1-fake news
#0-real news

[1 0 1 ... 0 1 1]


In [24]:
b.shape

(20800,)

In [25]:
#values are in text form but we know that computer cannot understand text so we need to convert to binary form
vectorizer=TfidfVectorizer()#Tf-Term frequency-counts how many times a word is repeated,and if word repeats many times it indicates the importance of that word
#idf-inverse document frequency,sometimes the most repeated word plays no role ,this finds those words.
vectorizer.fit(a)# converts to numbers(feature vectors)
a=vectorizer.transform(a)#transforms to respective features

In [26]:
print(a)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [27]:
#Splitting the dataset to train and test data
#random-state=split is same for the each time code is run
a_train,a_test,b_train,b_test=train_test_split(a,b,test_size=0.2,stratify=b,random_state=2)

In [28]:
#Training the (logistic regression) model
model=LogisticRegression()

In [29]:
model.fit(a_train,b_train)

LogisticRegression()

In [30]:
#Evaluation
#accuracy score on the training data
a_train_prediction=model.predict(a_train)#will predict a_train data
training_data_accuracy=accuracy_score(a_train_prediction,b_train)


In [31]:
print('Accuracy score of the training data',training_data_accuracy)

Accuracy score of the training data 0.9865985576923076


In [32]:
a_test_prediction=model.predict(a_test)
test_data_accuracy=accuracy_score(a_test_prediction,b_test)

In [33]:
print('Accuracy score of the test data',test_data_accuracy)

Accuracy score of the test data 0.9790865384615385


In [34]:
#Making predictive system
a_new=a_test[1]#1st test in our test column
prediction=model.predict(a_new)
print(prediction)
if(prediction[0]==0):
    print('News is real')
else:
    print('News is fake')

[0]
News is real


In [35]:
print(b_test[1])

0
